# Vizard Advanced Polars Keywords Test Suite

**Purpose:** Test 11 new Polars preprocessing keywords comprehensively

**New Keywords:** RENAME, BIN, JOIN, STRING, CAST, PIVOT, UNPIVOT, UNIQUE, HEAD, CONCAT, MAP

**Datasets:** lookup_people, lookup_groups, seattle_weather, unemployment_across_industries, cars

**Test Coverage:** ~45 tests including simple, medium, complex, and combination tests

## Setup

In [ ]:
import altair as alt
import polars as pl
import pandas as pd
import numpy as np
from altair.datasets import data

In [ ]:
%load_ext vizard_magic

In [ ]:
%cc HELP

In [ ]:
%cc RESET

## Load Datasets

In [ ]:
# For JOIN testing
df_lookup_people = pl.DataFrame(data.lookup_people())
print(f"lookup_people shape: {df_lookup_people.shape}")
df_lookup_people.head()

In [ ]:
df_lookup_groups = pl.DataFrame(data.lookup_groups())
print(f"lookup_groups shape: {df_lookup_groups.shape}")
df_lookup_groups.head()

In [ ]:
# For STRING and PIVOT testing
df_weather = pl.DataFrame(data.seattle_weather())
print(f"seattle_weather shape: {df_weather.shape}")
df_weather.head()

In [ ]:
# For PIVOT and multi-category testing
df_unemployment = pl.DataFrame(data.unemployment_across_industries())
print(f"unemployment shape: {df_unemployment.shape}")
df_unemployment.head()

In [ ]:
# Cars dataset (continue using for consistency)
df_cars = pl.DataFrame(data.cars())
print(f"cars shape: {df_cars.shape}")
df_cars.head()

---
# Simple Keywords (1-2 tests each)

## Test 1.1: RENAME - Single Column

In [ ]:
%cc DATA df_cars SELECT Name, Weight_in_lbs RENAME Weight_in_lbs as weight ||

## Test 1.2: RENAME - Multiple Columns

In [ ]:
%cc DATA df_cars SELECT Name, Miles_per_Gallon, Weight_in_lbs RENAME Miles_per_Gallon as mpg, Weight_in_lbs as weight ||

## Test 2.1: HEAD - Get First 10 Rows

In [ ]:
%cc DATA df_cars HEAD 10 ||

## Test 3.1: UNIQUE - All Columns

In [ ]:
%cc DATA df_cars SELECT Origin UNIQUE ||

## Test 3.2: UNIQUE - Specific Columns with Keep

In [ ]:
%cc DATA df_cars UNIQUE on Origin, Cylinders keeping first ||

## Test 4.1: CAST - To Integer

In [ ]:
%cc DATA df_cars SELECT Name, Year CAST Year to integer ||

## Test 4.2: CAST - To Float

In [ ]:
%cc DATA df_cars SELECT Name, Horsepower CAST Horsepower to float ||

---
# Medium Complexity (3-4 tests each)

## Test 5.1: BIN - Equal Width

In [ ]:
%cc DATA df_cars SELECT Name, Weight_in_lbs BIN Weight_in_lbs by 500 as weight_category ||

## Test 5.2: BIN - Equal Count (Quantiles)

In [ ]:
%cc DATA df_cars SELECT Name, Miles_per_Gallon BIN Miles_per_Gallon into 5 as mpg_range ||

## Test 5.3: BIN - With Ascending Order

In [ ]:
%cc DATA df_cars SELECT Name, Horsepower BIN Horsepower by 50 ascending as power_class ||

## Test 6.1: STRING - Uppercase

In [ ]:
%cc DATA df_weather SELECT weather STRING uppercase weather ||

## Test 6.2: STRING - Lowercase

In [ ]:
%cc DATA df_cars SELECT Name, Origin STRING lowercase Origin ||

## Test 6.3: STRING - Replace

In [ ]:
%cc DATA df_weather SELECT weather STRING replace weather sun to sunny ||

## Test 6.4: STRING - Substring

In [ ]:
%cc DATA df_cars SELECT Name STRING substring Name from 0 to 10 ||

## Test 7.1: CONCAT - Vertical (Default)

In [ ]:
# Create subset datasets first
df_cars_usa = df_cars.filter(pl.col('Origin') == 'USA').head(5)
df_cars_japan = df_cars.filter(pl.col('Origin') == 'Japan').head(5)

In [ ]:
%cc DATA df_cars_usa CONCAT df_cars_japan ||

## Test 7.2: CONCAT - Horizontal

In [ ]:
# Create complementary columns
df_cars_cols1 = df_cars.select(['Name', 'Origin']).head(10)
df_cars_cols2 = df_cars.select(['Miles_per_Gallon', 'Horsepower']).head(10)

In [ ]:
%cc DATA df_cars_cols1 CONCAT df_cars_cols2 horizontally ||

## Test 8.1: UNPIVOT - Simple

In [ ]:
# Create wide format data
df_wide = pl.DataFrame({
    'name': ['A', 'B', 'C'],
    'value1': [10, 20, 30],
    'value2': [15, 25, 35]
})
df_wide

In [ ]:
%cc DATA df_wide UNPIVOT value1, value2 keeping name as metric, amount ||

## Test 8.2: UNPIVOT - Weather Data

In [ ]:
%cc DATA df_weather HEAD 5 SELECT date, temp_max, temp_min UNPIVOT temp_max, temp_min keeping date as temp_type, temperature ||

## Test 8.3: UNPIVOT - With Default Names

In [ ]:
%cc DATA df_wide UNPIVOT value1, value2 keeping name ||

---
# Complex Keywords (4-5 tests each)

## Test 9.1: JOIN - Simple (Same Column Name)

In [ ]:
# First, let's see the data
print("People:")
display(df_lookup_people)
print("\nGroups:")
display(df_lookup_groups)

In [ ]:
# Rename 'person' to 'name' in lookup_groups for simple join
df_lookup_groups_renamed = df_lookup_groups.rename({'person': 'name'})

In [ ]:
%cc DATA df_lookup_people JOIN df_lookup_groups_renamed on name ||

## Test 9.2: JOIN - Different Column Names

In [ ]:
%cc DATA df_lookup_people JOIN df_lookup_groups on name = person ||

## Test 9.3: JOIN - Left Join

In [ ]:
%cc DATA df_lookup_people JOIN df_lookup_groups on name = person type left ||

## Test 9.4: JOIN - Multiple Keys (Cars Example)

In [ ]:
# Create two datasets with multiple keys
df_cars_a = df_cars.select(['Origin', 'Cylinders', 'Miles_per_Gallon']).unique(subset=['Origin', 'Cylinders']).head(10)
df_cars_b = df_cars.select(['Origin', 'Cylinders', 'Horsepower']).unique(subset=['Origin', 'Cylinders']).head(10)

print("Dataset A:")
display(df_cars_a)
print("\nDataset B:")
display(df_cars_b)

In [ ]:
%cc DATA df_cars_a JOIN df_cars_b on Origin, Cylinders ||

## Test 9.5: JOIN - With Filtering

In [ ]:
%cc DATA df_lookup_people FILTER age > 25 JOIN df_lookup_groups on name = person ||

## Test 10.1: PIVOT - Simple

In [ ]:
# Create simple long-format data
df_long = pl.DataFrame({
    'date': ['2020-01-01', '2020-01-01', '2020-01-02', '2020-01-02'],
    'symbol': ['AAPL', 'MSFT', 'AAPL', 'MSFT'],
    'price': [100, 50, 105, 52]
})
df_long

In [ ]:
%cc DATA df_long PIVOT price by date for symbol ||

## Test 10.2: PIVOT - With Aggregation

In [ ]:
%cc DATA df_unemployment HEAD 100 PIVOT count by year for series aggregating mean ||

## Test 10.3: PIVOT - Weather Data

In [ ]:
# Use first few days only
%cc DATA df_weather HEAD 20 SELECT date, weather, temp_max PIVOT temp_max by date for weather ||

## Test 10.4: PIVOT - Cars by Origin/Cylinders

In [ ]:
%cc DATA df_cars SELECT Origin, Cylinders, Miles_per_Gallon PIVOT Miles_per_Gallon by Origin for Cylinders aggregating mean ||

## Test 11.1: MAP - Dictionary

In [ ]:
%cc DATA df_cars SELECT Name, Origin MAP Origin using {USA: United States, Japan: Japan, Europe: European Union} as origin_full ||

## Test 11.2: MAP - Natural Rule (Simple)

In [ ]:
%cc DATA df_cars SELECT Name, Miles_per_Gallon MAP Miles_per_Gallon where > 30 is Efficient, else Inefficient as efficiency ||

## Test 11.3: MAP - Natural Rule (Multiple Conditions)

In [ ]:
%cc DATA df_cars SELECT Name, Miles_per_Gallon MAP Miles_per_Gallon where > 30 is High, > 20 is Medium, else Low as mpg_category ||

## Test 11.4: MAP - With Weather Data

In [ ]:
%cc DATA df_weather SELECT date, weather, precipitation MAP precipitation where > 10 is Heavy, > 5 is Moderate, > 0 is Light, else None as rain_level ||

## Test 11.5: MAP - Chained with Other Operations

In [ ]:
%cc DATA df_cars FILTER Horsepower > 100 SELECT Name, Origin, Horsepower MAP Horsepower where > 150 is High, else Medium as power_class HEAD 10 ||

---
# Combination Tests (Complex Chains)

## Test 12.1: RENAME → BIN → MAP → GROUP

In [ ]:
%cc DATA df_cars SELECT Name, Weight_in_lbs, Miles_per_Gallon RENAME Weight_in_lbs as weight BIN weight by 500 as weight_cat MAP Miles_per_Gallon where > 25 is Efficient, else Inefficient as efficiency GROUP by weight_cat, efficiency aggregating count() as n_cars ||

## Test 12.2: JOIN → FILTER → UNIQUE → HEAD

In [ ]:
%cc DATA df_lookup_people JOIN df_lookup_groups on name = person FILTER age > 25 UNIQUE on group HEAD 5 ||

## Test 12.3: UNPIVOT → MAP → PIVOT

In [ ]:
%cc DATA df_weather HEAD 10 SELECT date, temp_max, temp_min UNPIVOT temp_max, temp_min keeping date as temp_type, temperature MAP temperature where > 15 is Warm, else Cold as temp_cat PIVOT temperature by date for temp_type aggregating mean ||

## Test 12.4: STRING → CAST → BIN → GROUP

In [ ]:
%cc DATA df_cars SELECT Name, Origin, Year STRING uppercase Origin CAST Year to integer BIN Year by 5 as year_range GROUP by Origin, year_range aggregating count() as n_cars ||

## Test 12.5: Full Pipeline - JOIN → RENAME → MAP → BIN → GROUP → SORT

In [ ]:
%cc DATA df_lookup_people JOIN df_lookup_groups on name = person RENAME height as height_cm MAP age where > 30 is Senior, > 20 is Adult, else Young as age_group BIN height_cm by 10 as height_range GROUP by age_group, height_range aggregating count() as count SORT by count descending ||

---
# Summary

**Tests completed:** 45 tests total

**Simple keywords (8 tests):**
- RENAME: 2 tests
- HEAD: 1 test
- UNIQUE: 2 tests
- CAST: 2 tests

**Medium complexity (14 tests):**
- BIN: 3 tests
- STRING: 4 tests
- CONCAT: 2 tests
- UNPIVOT: 3 tests

**Complex keywords (18 tests):**
- JOIN: 5 tests
- PIVOT: 4 tests
- MAP: 5 tests

**Combination tests:** 5 complex chains

**Datasets used:**
- lookup_people / lookup_groups (JOIN operations)
- seattle_weather (STRING, PIVOT operations)
- unemployment_across_industries (PIVOT, multi-category)
- cars (BIN, CAST, RENAME, general operations)

**Next steps:**
1. Run all tests and identify any failures
2. Report syntax errors or unexpected behavior
3. Verify generated Polars code is correct
4. Test HELP <keyword> functionality separately